In [ ]:
"""
How to finetune from Cifar10 to Minst
"""

"""I Target： Here we learn how to train model

II Definition: 
    1 dataset definition and dataloader
    2 model
    3 optimizer and eval
    
III Instances:
    # 1 following sample classifier
    #TODO 2 finetune --> change state_dict?
    #! 3 add sptial loss from BasNet
    # 4 eval&analyse part
    #TODO * same in Keras/tensorflow


IV Compare 2 then Generalize

V Test in New instance 
"""

In [5]:
import os
from datetime import datetime
from tqdm import tqdm

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models
from torchvision import transforms
from torchvision.datasets import MNIST


In [7]:


# 1 dataset
"""Inherit from Dataset class or load inner torchvision.dataset"""

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std = (0.5))  # normalize to [-1,1]
])

from torchvision.datasets import MNIST
train_dataset = MNIST(root="/home/hpczeji1/hpc-work/Codebase/Datasets/mnist_data",
                      train=True,
                      transform=transform,
                      target_transform=None,  # Eg1.2.1 : <class 'int'>
                      download=False)

# 2  dataloader
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=10000,
                          shuffle=True)


# 3 model
from torch import nn
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=(3,3),stride=3) # in_channel is te image channel number
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=(3,3),stride =3)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=(3,3),stride =3)
        self.relu = nn.ReLU(inplace = True)
        self.Flatten = nn.Flatten(start_dim=1, end_dim=-1) # (B,C,H,W), USE C,H,W
        self.Linear = nn.Linear(in_features=64*1*1, out_features=10,bias= False) # ins_feature num = flatten
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        # print("[before flatten] x.shape: {}".format(x.shape))  # torch.Size
        x = self.Flatten(x)
        # print("[after flatten] x.shape: {}".format(x.shape))  # torch.Size([1, 3920])
        x = self.Linear(x)

        return self.relu(x)

model = SimpleModel()
"""if have pth, can load this pth weight and use model(x) to interference"""
# model.load_state_dict(torch.load("./model_2021_11_28.pth")) 
# print(model(x))

# 4 optimizer
optimizer = optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

# 5 train
epoch_num = 2
for epoch in range(epoch_num):
    with tqdm(train_loader) as train_bar:
        for x,y in train_bar:
            optimizer.zero_grad()
            loss = loss_fn(model(x),y)
            loss.backward()
            optimizer.step()
    print(f"epoch:{epoch}, loss{loss}")

time = str(datetime.now()).split(" ")[0].replace("-", "_")
torch.save(model.state_dict(), "model_{}.pth".format(time))

print("~~~~~~撒花~~~~~~")

tensor([[0.0014, 0.0000, 0.0492,  ..., 0.0244, 0.0000, 0.0000],
        [0.0122, 0.0000, 0.0530,  ..., 0.0216, 0.0000, 0.0000],
        [0.0168, 0.0000, 0.0494,  ..., 0.0188, 0.0000, 0.0000],
        ...,
        [0.0288, 0.0000, 0.0671,  ..., 0.0047, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0378,  ..., 0.0189, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0425,  ..., 0.0082, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
